In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter
from bioinfokit import analys, visuz
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
#### Mapping of samples to pops ####

pedigree = pd.read_csv("/expanse/projects/gymreklab/helia/TR_1000G/1000G.ped", delim_whitespace=True)
pedigree = pedigree[['SampleID','Superpopulation']]
samp_to_pop = pd.Series(pedigree.Superpopulation.values,index=pedigree.SampleID).to_dict()
H3Africa_names = pd.read_csv("/expanse/projects/gymreklab/helia/H3Africa/names/H3A_Baylor_sample_country.txt", header=None, delim_whitespace=True)

for index,row in H3Africa_names.iterrows():
    samp_to_pop[row[0]] = "H3Africa"

pop_count = pd.DataFrame(samp_to_pop.items(), columns=['sample', 'pop']).groupby("pop").count().reset_index().set_index('pop')['sample'].to_dict()
pop_count['BOTH_AFR'] = pop_count['AFR'] + pop_count['H3Africa']
pop_count['NON_AFR'] = pop_count['EAS'] + pop_count['SAS'] + pop_count['AMR'] + pop_count['EUR']



In [3]:
#### Extract gene information ####

genes = pd.read_csv("Homo_sapiens.GRCh38.108.gtf", sep = "\t", comment="#", header=None)
genes[0] = "chr" + genes[0].astype(str)


def find_genes(row):
    gene_df = genes[(genes[0] == row['chr']) & (genes[3] <= row['pos']) & 
                    (genes[4] >= row['pos'])]
    if len(gene_df) == 0:
        return None
    gene_df_info = gene_df[8].str.split(";")
    gene_set = set()
    for info in list(gene_df_info):
        for field in info:
            if "gene_name" in field:
                name = field.strip().replace("gene_name ","").replace('"','')
                gene_set.add(name)
    if len(gene_set) == 0:
        return None
                
    return ",".join(list(gene_set))

In [193]:
###### Computing expansion statistics for each locus ######

stats_df = pd.DataFrame(columns = ['chr', 'pos', 'period', 'motif', 
                                          'MAX_CN', 'pop', 'second_cn'])
for i in range(1,23):
    df = pd.read_csv(f"/expanse/projects/gymreklab/helia/ensembl/experiments/charact/diff_dist/diff/expansions_{i}.txt",
                    sep = "\t", header=None)
    df.columns = ['chr', 'pos', 'period', 'motif', 'MAX_CN', 'pop', 'second_cn']
    stats_df = pd.concat([stats_df, df])
    
for i in range(1,23):
    stats_df[stats_df['chr'] == f"chr{i}"]
    
    
    
# stats = []
# loci = list(zip(results.chr, results.position))
# for i in tqdm(range(len(loci))):
#     find_pop_expansion(loci[i][0], loci[i][1])
    
    
# stats_df = pd.DataFrame(stats, columns = ['chr', 'pos', 'motif', 
#                                           'max_cn', 'max_pop', 'second_cn', 'more_5',
#                                           'more_10', 'more15', 'more_mid'])

# stats_df['dif_cn'] = stats_df['max_cn'] - stats_df['second_cn']
# stats_df.to_csv("expansion_stats.csv", index=False, sep = "\t")

# stats_df = pd.read_csv("expansion_stats.csv", sep = "\t")

# # Final table

# expansions = stats_df[(stats_df['dif_cn'] >= 15) & (stats_df['more_mid'] > 10)].sort_values('dif_cn')

# def aggregate_info(row):
#     expansion_dict = {}
#     mid = int((row['max_cn'] + row['second_cn'])/2)
#     expansion_dict[row['max_cn'] - 5] = row['more_5']
#     expansion_dict[row['max_cn'] - 10] = row['more_10']
#     expansion_dict[row['max_cn'] - 15] = row['more15']
#     expansion_dict[mid] = row['more_mid']
#     return expansion_dict
    
# expansions['gene'] = expansions.apply(lambda row: find_genes(row), axis = 1)
# expansions['size:number'] = expansions.apply(lambda row: aggregate_info(row), axis = 1)
# expansions = expansions.drop(columns = ['more_5', 'more_10', 'more15', 'more_mid', 'dif_cn', 'second_cn'])
# expansions.columns = ['chr', 'pos', 'period', 'MAX_CN', 'pop', 'gene', 'size:number']

,chr,pos,period,motif,MAX_CN,pop,second_cn
0,chr10,5780217,4,AAAC,22,H3Africa,0
1,chr10,6534768,2,AT,29,AFR,13
2,chr10,8232972,1,A,16,SAS,0
3,chr10,8727582,2,AC,20,EAS,2
4,chr10,10786885,2,AC,29,H3Africa,4
...,...,...,...,...,...,...,...
31,chr10,116668924,2,AC,34,AMR,14
32,chr10,125049041,2,AC,58,SAS,41
33,chr10,127934024,2,AC,62,EUR,37
34,chr10,130477902,1,A,23,SAS,1


In [21]:
#### Code for examining found expansions one by one ####

def plot(chrom, pos): # Plot distributions for each population in one plot
    
    pops = ['AMR', 'AFR', 'EAS', 'EUR', 'SAS']

    # Get diffs for each population and make dataframe
    diff_data = []
    for pop in pops:
        addr = f"/expanse/projects/gymreklab/helia/ensembl/experiments/charact/diff_dist/diff/{pop}_{chrom}_diff.txt"
        x = !grep $pos $addr
        x = x[0].split("\t")
        assert(int(x[1]) == pos)
        diffs = [l.split(":") for l in x[3:]]
        diffs = [d[1].split("/") for d in diffs]
        diffs_alleles = []
        for d in diffs:
            if d[0] == ".":
                continue
            diffs_alleles.append(int(d[0]) / int(x[2]))
            diffs_alleles.append(int(d[1]) / int(x[2]))
        print(len(diffs_alleles), pop)
        if len(diffs_alleles) == 0:
            return -1
        diff_data.extend(
                [{'Super Population': pop, 'diff_n_bases': d} for d in diffs_alleles]
        )


    diff_data = pd.DataFrame(diff_data)
    diff_data['Copy Num. From Ref.'] = diff_data.diff_n_bases
    
    # Plot
    sns.displot(
        data=diff_data,
        x='Copy Num. From Ref.',
        hue='Super Population',
        kind="kde",
        common_norm=False,
        common_grid=True,
        bw_adjust=1.5
    )
    plt.suptitle(f"STR Copy Number Distribution for {chrom}:{pos}")
    plt.tight_layout()
    plt.show()

def examine(chrom, pos, threshold): # See how many expansions above threshold each population has for a locus

    # Get STR data
    if chrom not in str_data_cache:
        print(chrom)
        str_data = pd.read_csv(os.path.join(data_dir_path, f'{chrom}.csv'))
        str_data_cache[chrom] = str_data.drop_duplicates()
    str_data = str_data_cache[chrom]
    example_data = str_data[str_data.position == pos].iloc[0]
    
    motif_len = example_data.motif_len
    print(chrom, pos, motif_len)
    diffs_dict = {}
    for pop in ['AMR', 'AFR', 'EAS', 'EUR', 'SAS']:
        diffs = example_data[f'diffs_{pop}'].strip(' []').split(',')
        if diffs == ['']:
            return np.nan
        diffs = [int(int(d.strip()) / motif_len) for d in diffs]
        diffs_dict[pop] = len([d for d in diffs if d > threshold])
    return diffs_dict    

In [112]:
### Second method for finding expansions ###

# Load files for expansions above 40
dir_addr = "/expanse/projects/gymreklab/helia/ensembl/experiments/charact/diff_dist/large_expansions"
expansion_df = pd.DataFrame(columns = ['chr', 'pos', 'period', 'ru', 'sample', 'GB[0]', 'GB[1]', 'pop'])

for pop in ['EAS', 'SAS', 'EUR', 'AFR', 'AMR', 'H3Africa']:
    for chrom in range(1,23):
        df = pd.read_csv(f"{dir_addr}/large_expansion_{pop}_{chrom}.txt", sep = "\t", header=None)
        df.columns = ['chr', 'pos', 'period', 'ru', 'sample', 'GB[0]', 'GB[1]']
        if pop == "AFR" or pop == "H3Africa":
            df['pop'] = "BOTH_AFR"
        else:
            df['pop'] = "NON_AFR"
        #df['pop'] = pop
        expansion_df = pd.concat([expansion_df, df])

In [116]:
### Extract num_called ####

pop = "AFR"
chrom = 21

all_numcalled = pd.DataFrame(columns = ['chrom', 'start', 'numcalled-1', 'pop'])
for chrom in range(1,23):
    for pop in ['EAS', 'SAS', 'EUR', 'AFR', 'AMR', 'H3Africa']:
        df = pd.read_csv(f"/expanse/projects/gymreklab/helia/ensembl/experiments/allele_freq/freqs/freqs_chr{chrom}_{pop}.tab", 
                        sep = "\t")
        
        expansions = expansion_df[expansion_df['chr'] == f"chr{chrom}"]
        df = pd.merge(df, expansions, left_on = ['chrom', 'start'], right_on = ['chr', 'pos'], how='right')
        df = df[['chr', 'pos', 'numcalled-1']]
        df = df.fillna(pop_count[pop])
        df = df.drop_duplicates(subset=['chr', 'pos'])
        df['pop'] = pop
        all_numcalled = pd.concat([all_numcalled, df])
all_numcalled = all_numcalled.groupby(['chr', 'pos', 'pop'])['numcalled-1'].apply(list).to_dict()

all_numcalled_2 = {}
for key in all_numcalled:
    all_numcalled_2[(key[0], key[1], 'BOTH_AFR')] = all_numcalled[(key[0], key[1], 'AFR')][0] + all_numcalled[(key[0], key[1], 'H3Africa')][0]
    all_numcalled_2[(key[0], key[1], 'NON_AFR')] = all_numcalled[(key[0], key[1], 'EAS')][0] + \
                                                 all_numcalled[(key[0], key[1], 'EUR')][0] + \
                                                 all_numcalled[(key[0], key[1], 'AMR')][0] + \
                                                 all_numcalled[(key[0], key[1], 'SAS')][0]


In [118]:
### Find significant expansions ###

def find_sig_specific_expansions(threshold, df):
    
    # Filter those genotypes with at least allele expansion above threshold
    df['CN[0]'] = df['GB[0]'] / df['period']
    df['CN[1]'] = df['GB[1]'] / df['period']
    
    df = df[(df['CN[0]'] > threshold) | (df['CN[1]'] > threshold)]
    
    # Count how many alleles per genotype had expansion above threshold
    df['above_threshold'] = [x1 + x2 for x1, x2 in zip(list(df['CN[0]'] > threshold),
                                                list(df['CN[1]'] > threshold))]
        
    # Group by 
    by_pop = df.groupby(['chr', 'pos', 'pop', 'period', 'ru'], as_index=False).agg({'above_threshold':sum, 
                                                                              'CN[0]':np.max, 'CN[1]':np.max})
    # Maximum copy number per locus and population
    by_pop['MAX_CN'] = by_pop[["CN[0]", "CN[1]"]].max(axis=1).astype(int)
    by_pop = by_pop.drop(columns = ['CN[0]', 'CN[1]']) 
    by_pop.columns = ['chr', 'pos', 'pop', 'period', 'ru', '#alleles', 'MAX_CN']
    by_pop_dict = by_pop.groupby(['chr', 'pos', 
                                  'period', 'ru'])[['pop', '#alleles', 
                                              'MAX_CN']].apply(lambda g: g.values.tolist()).to_dict()
    significant_expansions = []

    for key in by_pop_dict: # Check if an expansion happened at least 10 times in 1 population
        expansion_list = by_pop_dict[key]
        max_freq = max([expansion[1] / all_numcalled_2[(key[0], key[1], expansion[0])] for expansion in expansion_list])
        if max_freq < 0.05:
            continue
        cnt = 0
        expanded_pop = ""
        for expansion in expansion_list:
            if (expansion[1] / all_numcalled_2[(key[0], key[1], expansion[0])]) > max_freq / 5:
                expanded_pop = expansion[0]
                cnt += 1
        if cnt < 2:
            significant_expansions.append([key[0], key[1], key[2], key[3], expanded_pop])
            
    significant_specific_expansions_poses = pd.DataFrame(significant_expansions, 
                                                         columns = ['chr', 'pos', 'period', 'ru', 'pop'])

    sig_spec_df = pd.merge(by_pop, significant_specific_expansions_poses, on = ['chr', 'pos', 'pop', 'period', 'ru'])
    return sig_spec_df

all_expansions = pd.DataFrame(columns = ['chr', 'pos', 'pop', 'period', 'ru', '#alleles', 'expansion_size'])
for i in range(4,11):
    expansion_size = i * 10
    above_expansion_size = find_sig_specific_expansions(expansion_size, expansion_df)
    above_expansion_size['expansion_size'] = expansion_size
    all_expansions = pd.concat([all_expansions, above_expansion_size])
all_expansions

,chr,pos,pop,period,ru,#alleles,expansion_size,MAX_CN
0,chr1,15374849,BOTH_AFR,1,A,61,40,56.0
1,chr1,35770419,BOTH_AFR,2,AT,64,40,57.0
2,chr1,81055684,BOTH_AFR,2,AT,121,40,62.0
3,chr1,93790613,BOTH_AFR,1,A,65,40,63.0
4,chr1,171426068,BOTH_AFR,1,A,69,40,62.0
5,chr1,210326921,BOTH_AFR,2,AT,548,40,94.0
6,chr1,216516625,BOTH_AFR,2,AC,73,40,57.0
7,chr1,244933753,BOTH_AFR,1,A,66,40,69.0
8,chr10,96940301,BOTH_AFR,1,A,88,40,74.0
9,chr10,98960702,BOTH_AFR,1,A,270,40,128.0


In [119]:
# group by threshold
grouped_expansions = all_expansions.groupby(['chr', 'pos', 'pop', 'period','ru', 'MAX_CN'])[('expansion_size', '#alleles')].apply(lambda g: g.values.tolist()).to_dict()
found_expansions = all_expansions.drop_duplicates(['chr', 'pos', 'pop', 'ru', 'period', 'MAX_CN']).sort_values(['chr', 'pos', 'pop'])
expansions_dicts = []
for expansion in grouped_expansions.values():
    expansion_dict = {}
    for pair in expansion:
        expansion_dict[pair[0]] = pair[1]
    expansions_dicts.append(expansion_dict)
found_expansions['size:number'] = expansions_dicts
found_expansions = found_expansions.drop(['#alleles', 'expansion_size'], axis = 1)
found_expansions['chr'] =  found_expansions['chr'].astype(str)
found_expansions['gene'] = found_expansions.apply(lambda row: find_genes(row), axis = 1)
found_expansions

,chr,pos,pop,period,ru,MAX_CN,size:number,gene
0,chr1,15374849,BOTH_AFR,1,A,56.0,{40: 61},FHAD1
1,chr1,35770419,BOTH_AFR,2,AT,57.0,{40: 64},None
0,chr1,65791915,BOTH_AFR,2,AG,87.0,{60: 92},None
2,chr1,81055684,BOTH_AFR,2,AT,62.0,{40: 121},None
3,chr1,93790613,BOTH_AFR,1,A,63.0,{40: 65},BCAR3
4,chr1,171426068,BOTH_AFR,1,A,62.0,{40: 69},None
0,chr1,185505112,BOTH_AFR,2,AT,69.0,{50: 74},None
5,chr1,210326921,BOTH_AFR,2,AT,94.0,{40: 548},None
6,chr1,216516625,BOTH_AFR,2,AC,57.0,{40: 73},ESRRG
7,chr1,244933753,BOTH_AFR,1,A,69.0,{40: 66},None


In [121]:
### Aggregate information from both methods ####

pd.set_option('display.max_rows', 600)
found_expansions.sort_values(["pop",'chr','pos']).reset_index()
found_expansions.to_csv("method2_2.csv", sep=",", index=False)
# both = pd.concat([found_expansions, 
#                   expansions]).reset_index(drop=True).drop_duplicates(subset = ['chr','pos',
#                                                                                 'pop','period']).sort_values(['chr', 'pos'], ignore_index=True)
# both.to_csv("Large_Expansions.csv", sep = ",", index = False)